In [ ]:
#Подключаем все нужные нам библеотеки
%matplotlib inline
from reportlab.pdfgen import canvas
from reportlab.lib import pagesizes
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.utils import ImageReader
from PyPDF2 import PdfFileMerger, PdfFileReader
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#создаём функцию, для извлечения название района, из поля District
#функция берёт значение серии ObjectAddress, находить в этом значении поля District
#приводить это поля к списку, и возвращать первое значение из этого списка
#по факту будет вычлениять первый найденный District из словоря ObjectAddress
def extract_district (x):
    return list(map(lambda a: a["District"], x))[0]
#получаем нужные данные, с помощью запроса с методом get
r = requests.get("https://video.ittensive.com/python-advanced/data-7361-2019-11-28.utf.json")
#полученныые данные передаём во фрейм, результат разбора json-файла
#также заполняем нулями отсутствующие значения
data = pd.DataFrame(json.loads(r.content)).fillna(value=0)
#для группировки по районам, нам нужно извлечь название района из поля District
#которое является, json разобранным в словарь
data["District"] = data["ObjectAddress"].apply(extract_district)
#после получения района для всезх значений данных, можно сгрупировать по нему
#и отсортировать по часу посетителей библиотек, в порядке убывания
data_sum = data.groupby("District").sum().sort_values("NumOfVisitors",
                                                     ascending=False)
#создаём холст                                                    
fig = plt.figure(figsize=(11,6))
#наносим на него подобласть
area = fig.add_subplot(1, 1, 1)
#строим результирующую диаграмму из 20 самых популярных районов
#для большей читаемости уберём районы из подписи на диаграмме
#задав пустые labels ровно по числу наших районов
data_sum[0:20]["NumOfVisitors"].plot.pie(ax = area,
                                        labels=[""]*20,
                                        label="Посещаемость",
                                        cmap="tab20")
#переносим все наши районы в легенду, это будет индекс из нашего набора данных

plt.legend(data_sum[0:20].index,
        #легенду выведем справа от диаграммы
        bbox_to_anchor=(1.5,1,0.1,0))
#сохраняем график в файл, для дальнейшей вставки в наш отчёт
plt.savefig("E:/ittensive/assets/readers.png")
#начинаем формирования отчёта, задаём шрифты
pdfmetrics.registerFont(TTFont("Trebuchet", "E:/ittensive/assets/Trebuchet.ttf"))
#задаём холст с  размерами
PDF = canvas.Canvas("E:/ittensive/assets/readers.pdf", pagesize=pagesizes.A4)
PDF.setFont("Trebuchet", 48)
#наносим на холст данные, 
PDF.drawString(70, 650, "Посетители библиотек")
PDF.drawString(80, 590, "по районам Москвы")
PDF.setFont("Trebuchet", 13)
#создаём вторую страницу отчёта
PDF.drawString(550, 820, "2")
#вставляем диаграмму, в виде отдельного изображения
#из того файла, что мы сохранили ранее
PDF.drawImage(ImageReader("E:/ittensive/assets/readers.png"), -200, 150)
PDF.setFont("Trebuchet", 20)
#дополнительно выедем информацию по самому популярному району
PDF.drawString(100, 150, "Самый популярный район")
PDF.setFont("Trebuchet", 24)
#это первый район в отсортированном списке
#выводим первое число посетителей в нашем кортеже
PDF.drawString(100, 120, data_sum.index.get_values()[0])
PDF.setFont("Trebuchet", 20)
PDF.drawString(100, 90,
               "Посетителей: " + str(int(data_sum["NumOfVisitors"].values[0])))
PDF.save()
#объединяем сгенерированный отчёт, и титульную страницу через PdfFileMerger()
files = ["E:/ittensive/assets/title.pdf", "E:/ittensive/assets/readers.pdf"]
merger = PdfFileMerger()
for filename in files:
    merger.append(PdfFileReader(open(filename, "rb")))
#создаём PDF файл report.pdf
merger.write("E:/ittensive/assets/report.pdf")